# Saverlife Chatbot Demo
### Write a question and hit enter

In [1]:
import ipywidgets as widgets
from fastai.text.all import *
import urllib.request

In [3]:
def _end_headline(x):
    if type(x) != str: return ''
    if x[-1] in ['.', '?', '!', '…', ':', '“', '”']: return x + ' '
    else: return x + '. '

def prep_text(file = 'text-1-15c.csv'):
    df = pd.read_csv(file)
    df = df[~df.text.isna()].copy()
    df = df[['headline', 'segment', 'subheading', 'text',
                               'top_tag', 'url', 'text length']].copy()
    df['joincol'] = df['subheading']
    df.loc[df.joincol=='','joincol'] = df.loc[df.joincol=='','headline']
    df.joincol = df.joincol.map(_end_headline).values
    df = df[df.headline.map(lambda x: x[:33]) != "Ask an Expert episode 2 follow-up"]
    return(df)

def breakup_text(text, max=90):
    if len(text) < max:
        print(text)
    else:
        for i in range(max-1,1,-1):
            if text[i] == ' ':
                print(text[:i])
                break
        breakup_text(text[i:].strip())

In [5]:
MODEL_URL = "https://www.dropbox.com/s/tmu82r4pd9h0ek8/chat_model.pkl?dl=1"

urllib.request.urlretrieve(MODEL_URL, "model.pkl")

learn = load_learner("model.pkl")

sdf = pd.read_pickle('segment_df.pkl')

In [6]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def chat_response(text, n):
    pred = learn.predict(text)
    df = pd.DataFrame(pred[2], columns = ['pct'])
    df['label_idx'] = df.index
    df = df.sort_values('pct', ascending=False).iloc[:50]
    url_df = pd.DataFrame(learn.dls.vocab[1])
    url_df['label_idx'] = url_df.index
    df = df.merge(url_df)
    df.columns = ['pct', 'idx', 'joincol']
    df = df.merge(sdf)
    df['score'] = df['pct'] / (df['text length'] + 50) * 10000
    df = df.sort_values('score', ascending=False)
    print(f'{color.RED}Here are our top {n} results:{color.END}')
    for i in range(0,n):
        print(color.BOLD + color.UNDERLINE)
        breakup_text(df.iloc[i]['headline'])
        print(color.END + color.BOLD + color.DARKCYAN)
        breakup_text(df.iloc[i]['subheading'])
        print(color.END)
        breakup_text(df.iloc[i]['text'])
        print(f"\n{color.GREEN}Read the full article on the SaverLife Blog")
        print(df.iloc[i]['url'])
        print(color.END)

In [13]:
textbox = widgets.Text(
    value='How do I save more?',
    placeholder='Write your quetion here!',
    description='Question:',
    disabled=False,
    continuous_update=False
)

slider = widgets.IntSlider(
    description='Responses', max=20, min=1, value=3, continuous_update=False)


out = widgets.interactive_output(chat_response, {'text': textbox, 'n': slider})

widgets.HBox([widgets.VBox([textbox, slider])])

Here are our top 3 results:

“Pay Yourself First” – Your Savings Mantra

Focus on You. Pay Yourself First.

Have you ever thought, “I’ll pay all of my bills and save what’s left”? Well, how much
was left? When we wait to save until the end of the month, we usually learn that we have
more month than money and there is nothing left. We challenge you to commit to paying
yourself first. “Pay yourself first.” We hear it all the time, but what does it really
mean? Most of us have thought about saving more or saving more regularly, but how many of
us really save before paying our bills?

Read the full article on the SaverLife Blog
https://www.saverlife.org/budgeting/pay-first-savings-mantra/


What does saving mean to you?

Saving for Soon

Earlier this year, we ran a campaign called Savers Win, where SaverLife members pledged
to save their tax refunds and won prizes. If you participated in Savers Win, you may have
answered a survey from us about whether you actually did save your refund. Whe

In [14]:
widget = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ '[variable containing long text goes here]' +' </p>')

In [15]:
widgets.HBox([out])